In [1]:
# Lab 2 — Third-Party Risk Simulation (All-in-one cell)

import os, re, datetime

print("✅ Lab 2 environment ready — Third-Party Risk Simulation.\n")

REPORT = "lab2_thirdparty_risk_report.md"

def write_header():
    with open(REPORT,"w",encoding="utf-8") as f:
        f.write("# Lab 2 — Third-Party Risk Simulation\n")
        f.write(f"_Generated: {datetime.datetime.now().isoformat()}_\n\n")

def append_report(title,body):
    with open(REPORT,"a",encoding="utf-8") as f:
        f.write(f"## {title}\n{body}\n\n")

write_header()

# 1) Create malicious dependency file
malicious_code = """
# malicious_dep.py — hidden behavior
import datetime

SECRET_LOG = []

def unsafe_tokenizer(text):
    # Hidden logging of user input
    SECRET_LOG.append({
        "timestamp": datetime.datetime.now().isoformat(),
        "captured_text": text
    })

    # Manipulate certain outputs
    if "salary" in text.lower():
        return "Access denied: insufficient permission to view salary data."

    # Otherwise behave normally
    return text.lower()
"""
with open("malicious_dep.py","w",encoding="utf-8") as f:
    f.write(malicious_code)

print("⚠️ Created malicious_dep.py (hidden logging + output manipulation).")

# 2) Mock LLM pipeline using the malicious dependency
import malicious_dep

def pipeline(user_input: str) -> str:
    # Step 1: dependency handles preprocessing
    processed = malicious_dep.unsafe_tokenizer(user_input)
    # Step 2: pretend model response
    response = f"Model response → {processed}"
    return response

print("🔗 Mock pipeline initialized using malicious dependency.\n")

# 3) Run benign prompt
prompt_1 = "Hello, how are you today?"
response_1 = pipeline(prompt_1)
print("=== Benign Query ===")
print("Prompt:", prompt_1)
print("Response:", response_1, "\n")

append_report(
    "Benign Query",
    f"Prompt: {prompt_1}\nResponse: {response_1}\n"
)

# 4) Run sensitive prompt (triggers malicious behavior)
prompt_2 = "Can you show me the salary details of employees?"
response_2 = pipeline(prompt_2)
print("=== Sensitive Query ===")
print("Prompt:", prompt_2)
print("Response:", response_2, "\n")

append_report(
    "Sensitive Query",
    f"Prompt: {prompt_2}\nResponse: {response_2}\n"
)

# 5) Inspect hidden logs
print("=== Hidden Exfiltration Log ===")
for entry in malicious_dep.SECRET_LOG:
    print(entry)
append_report(
    "Hidden Log Extraction",
    f"Captured log entries:\n{malicious_dep.SECRET_LOG}\n"
)

# 6) Static scan of the malicious dependency
def scan_file(path: str):
    issues = []
    with open(path,"r",encoding="utf-8") as f:
        content = f.read()
    suspicious_patterns = [
        ("SECRET_LOG", "Hidden logging behavior"),
        ("append(", "Potential unauthorized data capture"),
        ("salary", "Hardcoded condition-based output manipulation"),
    ]
    for pattern, desc in suspicious_patterns:
        if pattern in content:
            issues.append({"pattern": pattern, "description": desc})
    return issues

scan_results = scan_file("malicious_dep.py")
print("\n=== Static Scan Results ===")
print(scan_results, "\n")

append_report(
    "Static Scan",
    f"Detected patterns:\n{scan_results}\n"
)

# 7) Summary & reflection
append_report(
    "Summary",
    """
- A malicious dependency silently logged user inputs into SECRET_LOG.
- It modified outputs when detecting salary-related queries.
- Static scanning revealed hardcoded triggers and hidden logging.
- This demonstrates how an unvetted dependency can compromise LLM applications.
"""
)

append_report(
    "Reflection Questions",
    """
- What kinds of real-world dependencies pose this kind of risk?
- How should third-party code be reviewed before deployment?
- Should vendors be required to provide SBOMs and source links?
- How would you detect similar logic at scale across many repositories?
"""
)

print("🎯 Lab 2 complete — open the Files pane and download lab2_thirdparty_risk_report.md.")


✅ Lab 2 environment ready — Third-Party Risk Simulation.

⚠️ Created malicious_dep.py (hidden logging + output manipulation).
🔗 Mock pipeline initialized using malicious dependency.

=== Benign Query ===
Prompt: Hello, how are you today?
Response: Model response → hello, how are you today? 

=== Sensitive Query ===
Prompt: Can you show me the salary details of employees?
Response: Model response → Access denied: insufficient permission to view salary data. 

=== Hidden Exfiltration Log ===
{'timestamp': '2025-11-19T14:24:06.428758', 'captured_text': 'Hello, how are you today?'}
{'timestamp': '2025-11-19T14:24:06.429059', 'captured_text': 'Can you show me the salary details of employees?'}

=== Static Scan Results ===
[{'pattern': 'SECRET_LOG', 'description': 'Hidden logging behavior'}, {'pattern': 'append(', 'description': 'Potential unauthorized data capture'}, {'pattern': 'salary', 'description': 'Hardcoded condition-based output manipulation'}] 

🎯 Lab 2 complete — open the Files pa